In [56]:
import pandas as pd
import numpy as np
import re
import time
# https://www.kaggle.com/ceshine/1000x-faster-data-manipulation/notebook

In [2]:
pd.set_option('max_columns', 15)
pd.set_option('chained_assignment', None)

In [3]:
df = pd.read_csv('sample_data_pygotham2019.csv', 
                 parse_dates=['Date Created', 'Original Record: Date Created'])

In [4]:
df.shape

(273399, 14)

In [5]:
df.head(5)

,Internal ID,ID,Category,Lead Source,Lead Source Category,Current Status,Status at Time of Lead,Original Record: Date Created,Date Created,Inactive,Specialty,Providers,duplicate_leads,bad_leads
0,16,1966,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-08-14 15:18:00,No,Dermatology,0.0,False,True
1,16,6213,Practice,Nonpaid : Email : Webinar : Efficiency Kiosk A...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-08-14 15:18:00,No,Dermatology,0.0,True,True
2,16,6936,Practice,Nonpaid : Webinar : AdvancedFax and Patient : ...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-12-03 17:33:00,No,Dermatology,0.0,False,True
3,16,4557,Practice,Nonpaid : Webinar : AdvancedFax and Patient : ...,- None -,CLIENT-Deactivated,- None -,2003-04-30 18:15:00,2015-12-03 17:33:00,No,Dermatology,0.0,True,True
4,20,8637,Practice,Nonpaid : Webinar : HIPAA Audit Prep : LSID 53...,- None -,CLIENT-Live,CLIENT-Live,2003-04-30 18:15:00,2016-08-24 14:09:00,No,Plastic Surgery,1.0,False,False


In [6]:
df.dtypes

Internal ID                               int64
ID                                        int64
Category                                 object
Lead Source                              object
Lead Source Category                     object
Current Status                           object
Status at Time of Lead                   object
Original Record: Date Created    datetime64[ns]
Date Created                     datetime64[ns]
Inactive                                 object
Specialty                                object
Providers                               float64
duplicate_leads                            bool
bad_leads                                  bool
dtype: object

In [7]:
def set_lead_status(row):
    if row['Current Status'] == '- None -':
        return row['Status at Time of Lead']
    else:
        return row['Current Status']

In [8]:
df_test = df.copy()
df_test.apply(set_lead_status, axis=1)

0           CLIENT-Deactivated
1           CLIENT-Deactivated
2           CLIENT-Deactivated
3           CLIENT-Deactivated
4                  CLIENT-Live
                  ...         
273394        PROSPECT-CURRENT
273395    LEAD-Contact Program
273396    LEAD-Contact Program
273397    LEAD-Contact Program
273398         LEAD-No Contact
Length: 273399, dtype: object

In [11]:
# Pandas Series Vectorized baby!!

# you can pass the output directly into a pandas Series

df['normalized_status'] = np.where(
    df['Status at Time of Lead'] == '- None -',  # <-- condition (W)
    df['Current Status'],                        # <-- return if true (T)
    df['Status at Time of Lead']                 # <-- return if false (F)
)

In [20]:
# df['normalized_status']

In [21]:
df['normalized_status'] = np.where(
    df['Status at Time of Lead'].values == '- None -',
    df['Current Status'].values, 
    df['Status at Time of Lead'].values
)

In [22]:
df['Status at Time of Lead'].values

array(['- None -', '- None -', '- None -', ..., 'LEAD-No Contact',
       'LEAD-No Contact', 'LEAD-No Contact'], dtype=object)

In [35]:
type(df['Current Status'].values)

numpy.ndarray

In [40]:
type(df['Current Status'].values[1])

str

In [41]:
type(df['Current Status'][0])

str

In [30]:
type(df['normalized_status'])

pandas.core.series.Series

# Multiple conditions

In [42]:
list1 = ['LEAD-3 Flame No Contact', 'LEAD-Campaign', 'LEAD-Claim', 'LEAD-Contact Program', 
         'LEAD-General Pool', 'LEAD-No Contact', 'LEAD-Nurture', 'LEAD-Unqualified', 'PROSPECT-LOST']

list2 = ['- None -', 'CLIENT-Closed-Sold', 'CLIENT-Handoff', 'CLIENT-Implementation', 'CLIENT-Implementation (EMR)',
         'CLIENT-Live', 'CLIENT-Partner', 'CLIENT-Referring Consultant', 'CLIENT-Transferred', 'LEAD-Engaged', 
         'LEAD-Long-Term Opportunity', 'PROSPECT-CURRENT', 'PROSPECT-LONG TERM', 'PROSPECT-NO DECISION']

# apply version
def lead_category(row):
    if row['Original Record: Date Created'] == row['Date Created']:
        return 'New Lead'
    elif row['normalized_status'].startswith('CLI'):
        return 'Client Lead'
    elif row['normalized_status'] in list1:
        return 'MTouch Lead'
    elif row['normalized_status'] in list2:
        return 'EMTouch Lead'
    return 'NA'

df['lead_category0'] = df.apply(lead_category, axis=1)
df['lead_category0']

0         Client Lead
1         Client Lead
2         Client Lead
3         Client Lead
4         Client Lead
             ...     
273394       New Lead
273395       New Lead
273396       New Lead
273397       New Lead
273398       New Lead
Name: lead_category0, Length: 273399, dtype: object

## nested np.where.... not so fun

In [43]:
df['lead_category'] = \
    np.where(df['Original Record: Date Created'].values == df['Date Created'].values, 'New Lead', 
            np.where(df['normalized_status'].str.startswith('CLI').values, 'Client Lead', 
                    np.where(df['normalized_status'].isin(list1).values, 'MTouch Lead', 
                            np.where(df['normalized_status'].isin(list2).values, 'EMTouch Lead', 
                                     'NA') 
                                  )
                         )
                )

# Enter numpy.select()

In [ ]:
# Order of operations matter!!!
conditions = [
    df['Original Record: Date Created'].values == df['Date Created'].values,
    df['normalized_status'].str.startswith('CLI').values,
    df['normalized_status'].isin(list1).values,
    df['normalized_status'].isin(list2).values
]

choices = [
    'New Lead', 
    'Client Lead', 
    'MTouch Lead',
    'EMTouch Lead'
]


df['lead_category1'] = np.select(conditions, choices, default='NA')  

In [ ]:
#nested: 
# With np.select, could do .values here for additional speed, but left out to avoid too much text
conditions = [
    ((df['Inactive'] == 'No') & (df['Providers'] == 0)),
    ((df['Inactive'] == 'No') & (df['Providers'] < 5)),
    df['Inactive'] == 'No',
    df['duplicate_leads'],  # <-- you can also just evaluate boolean arrays
    df['bad_leads'],
]

choices = [
    'active_no_providers',
    'active_small',
    'active_normal',
    'is_dup',
    'active_bad',
]

df['lead_type_vec'] = np.select(conditions, choices, default='NA')


## regex search, apply

In [44]:
# Doing a regex search to find string patterns

def find_paid_nonpaid(s):
    if re.search(r'non.*?paid', s, re.I):
        return 'non-paid'
    elif re.search(r'Buyerzone|^paid\s+', s, re.I):
        return 'paid'
    else:
        return 'unknown'

In [45]:
df['lead_source_paid_unpaid'] = df['Lead Source'].apply(find_paid_nonpaid)

## regex with np.select

In [46]:
conditions = [
    df['Lead Source'].str.contains(r'non.*?paid', case=False, na=False),
    df['Lead Source'].str.contains(r'Buyerzone|^paid\s+', case=False, na=False),
]

choices = [
    'non-paid',
    'paid'
]

df['lead_source_paid_unpaid1'] = np.select(conditions, choices, default='unknown')

# Dictionary lookups

In [47]:
channel_dict = {
    'Billing Service': 'BS', 'Consultant': 'PD', 'Educational': 'PD', 
    'Enterprise': 'PD', 'Hospital': 'PD', 'IPA': 'PD', 'MBS': 'RCM', 
    'MSO': 'PD', 'Medical practice': 'PD', 'Other': 'PD', 'Partner': 'PD',
    'PhyBillers': 'BS', 'Practice': 'PD', 'Purchasing Group': 'PD',
    'Reseller': 'BS', 'Vendor': 'PD', '_Other': 'PD', 'RCM': 'RCM'
}

def a_dict_lookup(row):
    if row['Providers'] > 7:
        return 'Upmarket'
    else:
        channel = channel_dict.get(row['Category'])
        return channel

In [48]:
df['dict_lookup'] = df.apply(a_dict_lookup, axis=1)

## np.where, .map()

In [49]:
df['dict_lookup1'] = np.where(
    df['Providers'].values > 7,              # W
    'Upmarket',                              # True
    df['Category'].map(channel_dict)         # False, map to dict?
)

In [50]:
channel_values = df['Category'].map(channel_dict)
df['dict_lookup1'] = np.where(
    df['Providers'] > 7, 
    'Upmarket',
    channel_values
)

In [52]:
# # dates:
# wtc2 = np.where(
#     df['Start Date'].isnull().values,
#     (df['Original Record: Date Created'].values - df['Date Created']).dt.days / 7,
#     (df['Date Created'].values - df['Start Date']).dt.days / 7
# )

# pandas .shift, np.select

In [54]:
previous_id = df['Internal ID'].shift(1).fillna(0).astype(int)
previous_date = df['Original Record: Date Created'].shift(1).fillna(pd.Timestamp('1900'))

In [55]:
conditions = [
    ((df['Internal ID'].values ==  previous_id) & 
     (df['Date Created'] - previous_date).astype('timedelta64[D]') < 5),
    df['Internal ID'].values ==  previous_id
]
choices = [0, 1]
df['time_col1'] = np.select(conditions, choices, default=1)